#Reading data

In [ ]:
import torch
from spacy.tokenizer import Tokenizer
from torchtext import data
from torchtext import datasets

SEED = 11

torch.manual_seed(SEED)                         ## Reproducibility
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)   ## Text field
LABEL = data.LabelField(dtype = torch.float)                    ## Label Field

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
import random
test_data, valid_data = test_data.split(random_state = random.seed(SEED))

In [ ]:
print(len(train_data), len(valid_data), len(test_data))

25000 7500 17500


In [ ]:
## Let's create 60 000 length vocabulary
MAX_VOCAB_SIZE = 60000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", ## Global Vectors for Word Representation with 6B tokens and 100d
                 unk_init = torch.Tensor.normal_) ## normal distribution for out-of-vocab words

LABEL.build_vocab(train_data)

In [ ]:
print(f"Number of words in TEXT vocab: {len(TEXT.vocab)}")
print(f"Number of words in LABEL vocab: {len(LABEL.vocab)}")

Number of words in TEXT vocab: 60002
Number of words in LABEL vocab: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(10))

[('the', 289838), (',', 275296), ('.', 236843), ('and', 156483), ('a', 156282), ('of', 144055), ('to', 133886), ('is', 109095), ('in', 87676), ('I', 77546)]


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')   ## Let's use GPU if abailable

## BucketIterator will help us to minimize padding the amount of padding per batch
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

#Creating the LSTM model

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, 
                 n_layers, bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers = n_layers, 
                           bidirectional = bidirectional, 
                           dropout = dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedding = self.embedding(text)    ## shape = (sent_length, batch_size)
        embedded = self.dropout(embedding)  ## shape = (sent_length, batch_size, emb_dim)
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)    ## pack sequence
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)        ## unpack sequence

        ## output shape = (sent_len, batch_size, hid_dim * num_directions)
        ## output over padding tokens are zero tensors
        
        ## hidden shape = (num_layers * num_directions, batch_size, hid_dim)
        ## cell shape = (num_layers * num_directions, batch_size, hid_dim)
        
        ## concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        ## and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) ## shape = (batch_size, hid_dim * num_directions)
            
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.4
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = Model(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

In [ ]:
train_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"There are {train_params} trainable parameters")

There are 8310857 trainable parameters


###Replace initial embedding with pretrained embedding

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.5108,  1.0283, -0.3532,  ..., -0.3283,  1.5016, -0.0413],
        [-1.7518, -2.9519,  0.0745,  ..., -1.8542,  0.2554, -0.1562],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.5132,  0.2418, -0.7890,  ...,  0.2582, -0.3415,  0.1267],
        [ 0.3138,  0.3586, -0.0497,  ...,  0.6948, -0.4473, -0.0625],
        [-0.3410,  1.2659, -0.7284,  ...,  0.1234,  0.9430,  0.2376]])

### Replace <unk> and <pad> with zeros (they were initialized with the normal distribution)

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.5132,  0.2418, -0.7890,  ...,  0.2582, -0.3415,  0.1267],
        [ 0.3138,  0.3586, -0.0497,  ...,  0.6948, -0.4473, -0.0625],
        [-0.3410,  1.2659, -0.7284,  ...,  0.1234,  0.9430,  0.2376]])


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.train()
    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)
        accuracy = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

In [ ]:
# def binary_accuracy(preds, y):

#     rounded_preds = torch.round(torch.sigmoid(preds))
#     correct = (rounded_preds == y).float() #convert into float for division 
#     accuracy = correct.sum() / len(correct)
#     return accuracy
def binary_classification_metrics(prediction, ground_truth):
    '''
    Computes metrics for binary classification

    Arguments:
    prediction, np array of bool (num_samples) - model predictions
    ground_truth, np array of bool (num_samples) - true labels

    Returns:
    precision, recall, f1, accuracy - classification metrics
    '''

    prediction = torch.round(torch.sigmoid(prediction))
    correct = (prediction == ground_truth).float() #convert into float for division 
    
    precision = 0
    recall = 0
    accuracy = 0
    f1 = 0

    tp = 0      ## true positive
    tn = 0      ## true negative
    fp = 0      ## false positive
    fn = 0      ## false negative

    for i in range(len(prediction)):
        if prediction[i] == True and ground_truth[i] == True:
            tp += 1
        if prediction[i] == True and ground_truth[i] == False:
            fp += 1
        if prediction[i] == False and ground_truth[i] == True:
            fn += 1
        if prediction[i] == False and ground_truth[i] == False:
            tn += 1

    accuracy = (tp + tn)/(tp + tn + fp + fn)
    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    f1 = 2 * (precision * recall)/(precision + recall)

    return precision, recall, f1, accuracy

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:

            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            
            accuracy = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator), epoch_accuracy / len(iterator)

def metrics(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_f1 = 0

    tp = tn = fp = fn = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:

            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            
            precision, recall, f1, accuracy = binary_classification_metrics(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_f1 += f1
        
    return epoch_loss / len(iterator), epoch_f1 / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.0017)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)            ## use GPU
criterion = criterion.to(device)

In [ ]:
N_EPOCHS = 6

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_accuracy = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_accuracy = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model, 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_accuracy*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_accuracy*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 01 | Epoch Time: 0m 54s
	Train Loss: 0.621 | Train Acc: 64.96%
	 Val. Loss: 0.691 |  Val. Acc: 59.10%
Epoch: 02 | Epoch Time: 0m 54s
	Train Loss: 0.467 | Train Acc: 77.47%
	 Val. Loss: 0.320 |  Val. Acc: 86.69%
Epoch: 03 | Epoch Time: 0m 54s
	Train Loss: 0.252 | Train Acc: 90.01%
	 Val. Loss: 0.282 |  Val. Acc: 88.37%
Epoch: 04 | Epoch Time: 0m 54s
	Train Loss: 0.182 | Train Acc: 93.29%
	 Val. Loss: 0.290 |  Val. Acc: 88.64%
Epoch: 05 | Epoch Time: 0m 54s
	Train Loss: 0.135 | Train Acc: 95.15%
	 Val. Loss: 0.326 |  Val. Acc: 88.51%
Epoch: 06 | Epoch Time: 0m 54s
	Train Loss: 0.111 | Train Acc: 96.19%
	 Val. Loss: 0.337 |  Val. Acc: 88.09%


In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.347 | Test Acc: 87.92%


# Saving model and vocabulary

In [ ]:
## Use if you don't save your model during training
# torch.save(model, 'model.pt')

In [ ]:
def save_vocab(vocab, path):
    import pickle
    output = open(path, 'wb')
    pickle.dump(vocab, output)
    output.close()

save_vocab(TEXT.vocab, 'vocab.txt')

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


# Loading model and using for typical review

In [ ]:
import pickle

with open('vocab.txt', 'rb') as file:
    vocab = pickle.load(file)

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
sentence = "Shut the fuck up those who say that it is the bad film. It's amazing"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = torch.load('model.pt', map_location = device)

predict_sentiment(loaded_model, sentence)

0.5432041883468628

In [ ]:
test_loss, test_f1 = metrics(loaded_model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test F1: {test_f1*100:.2f}%')

Test Loss: 0.282 | Test Acc: 88.56%


# Additional

In [ ]:
from google.colab import files
files.download('model.pt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>